In [ ]:
%load_ext google.colab.data_table
from google.colab import drive 
drive.mount('/content/drive')
import os.path, time
print("created: %s" % time.ctime(os.path.getctime("/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Input/Data.json")))
!pip install transformers
import pandas as pd 
import numpy as np
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Mounted at /content/drive
created: Sun Jan  3 23:42:08 2021
     |████████████████████████████████| 1.5MB 4.3MB/s 
     |████████████████████████████████| 890kB 27.5MB/s 
     |████████████████████████████████| 2.9MB 18.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=141523aba536ba4ff6fcee6e6854926963de5395244e20e36f240d0d673ce9e7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import numpy as np
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
data = pd.read_json("/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Input/Data.json")


In [ ]:
%load_ext google.colab.data_table
df = pd.DataFrame({'art_id'	:data['art_id'], 
                            'art_tag':data['art_tag']})
train_set = df[df['art_tag'].notnull()]
tags=train_set['art_tag']
for i,row in enumerate(tags):
  try :
    tags[i]=row.replace("#","").replace('[','').replace(']','')
  except:
    tags[i]=row.replace("#","")
train_set['art_tag']=tags
print(len(train_set))
train_set


The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_n

10625


,art_id,art_tag
74,74,"'Evénement', 'Relation citoyens', 'Internet'"
75,75,"'Evénement', 'Relation citoyens', 'Internet'"
76,76,"'Evénement', 'Gestion eau', 'Mutualisation', '..."
77,77,"'Innover', 'Relation citoyens', 'Collaboratif'..."
78,78,"'Comptabilité', 'Dématérialisation'"
...,...,...
11639,11639,"['Numérique', 'et', 'communication']"
11640,11640,"['Environnement', 'et', 'énergie']"
11641,11641,"['Administration', 'et', 'ressources', 'humain..."
11642,11642,"['Habitat', 'et', 'logement']"


In [ ]:
 def tags_to_word(data):
  All_tags : list = []
  list_tags : list = []
  tags : list = []
  for tags in data["art_tag"]:
    if str(tags) == "Not Found" or pd.isnull(tags):
      pass
    elif type(tags)== str:
      try :
        list_tags.append(eval(tags.replace("#","").replace('[','').raplace(']','')))
      except:
        list_tags.append([tags.replace("#","")])
    else:
      print(type(tags),tags)
  splites_list=[]
  for sentence in list_tags:
    for tag in sentence:
      for i in range(len(tag.split())):
        splites_list.append(tag.split()[i])
  return splites_list


  return(list_tags)



In [ ]:
nltk.download('stopwords')
stopwords_fr = stopwords.words('french')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
vectorizer = TfidfVectorizer(stop_words=stopwords_fr)
X = vectorizer.fit_transform(train_set['art_tag'])
terms = vectorizer.get_feature_names()
print(vectorizer.vocabulary_)

{'evénement': 330, 'relation': 709, 'citoyens': 141, 'internet': 467, 'gestion': 391, 'eau': 271, 'mutualisation': 566, 'cloud': 151, 'innover': 453, 'collaboratif': 155, 'dématérialisation': 262, 'comptabilité': 170, 'anti': 44, 'virus': 895, 'sécurité': 814, 'messagerie': 543, 'communication': 168, 'finances': 359, 'intercommunalité': 463, 'services': 775, 'ligne': 504, 'app': 46, 'mobile': 548, 'elus': 285, 'mobilité': 551, 'aménagement': 38, 'territoire': 828, 'acteurs': 16, 'territoires': 829, 'action': 17, 'logement': 513, 'permis': 642, 'construire': 192, 'hiérarchie': 417, 'normes': 593, 'bâtiment': 112, 'politique': 655, 'ville': 890, 'copropriété': 199, 'montagne': 557, 'collectivités': 162, 'locales': 508, 'hébergement': 426, 'urgence': 870, 'coeur': 154, 'travaux': 848, 'décentralisation': 254, 'habitat': 410, 'indigne': 441, 'aides': 32, 'financières': 361, 'maison': 530, 'public': 691, 'msap': 562, 'rénovation': 748, 'énergétique': 946, 'construction': 191, 'social': 787,

In [ ]:
kmeans = KMeans(n_clusters= 11)
ytrain = kmeans.fit_predict(X)

In [ ]:
from scipy.spatial import distance
def get_tag_center_cluster(ytrain,k):
  dict_near_cloud={}
  list_tags=[]
  for i in set(ytrain):
    cluster = np.where(ytrain==i)
    X_cluster = X[cluster]
    min_distances =[distance.cosine(X_cluster[point].toarray(), kmeans.cluster_centers_[i]) for point in range(X_cluster.shape[0])]
    index_near_cloud = np.argpartition(min_distances, k)[:k]
    vect_min = sorted(min_distances)[:k]
    for j in index_near_cloud:
      tags =vectorizer.inverse_transform(X[j])
      tags = np.array(tags).tolist()
      for l in tags:
        for word in l:
          list_tags.append(word)
    dict_near_cloud[i]=list(set(list_tags))
    list_tags=[]
    index_near_cloud=[]
  return(dict_near_cloud)
    

get_tag_center_cluster(ytrain,5)

{0: ['19',
  'changemanagement',
  'found',
  'transformationdigitale',
  'expériencecollaborateur',
  'carrières',
  'talentmanagement',
  'actualités',
  'covid',
  'not'],
 1: ['économique',
  'santé',
  'protection',
  'education',
  'environnement',
  'publique',
  'mer',
  'vie',
  'recherche',
  'urbanisme',
  'territoires',
  'ville',
  'société',
  'innovation',
  'economie',
  'sociale'],
 2: ['actualités'],
 3: ['économique',
  'relations',
  'insertion',
  'education',
  'textile',
  'industrie',
  'coopération',
  'vie',
  'internationales',
  'recherche',
  'international',
  'professionnelle',
  'société',
  'travail',
  'emploi',
  'innovation',
  'economie',
  'sociale'],
 4: ['rh3',
  'talentmanagement',
  'education',
  'public',
  'changemanagement',
  'recherche',
  'société',
  'paie',
  'développement',
  'actualités',
  'secteur'],
 5: ['numérique',
  'économique',
  'relations',
  'santé',
  'protection',
  'sciences',
  'technologie',
  'structure',
  'entrepr